# 보이는 웹툰 화면의 회차 url 리스트화

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [2]:
index_page_url = "https://m.comic.naver.com/webtoon/list?titleId=747269&week=wed"

In [3]:
headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

In [4]:
res = requests.get(index_page_url, headers=headers)
res

<Response [200]>

In [5]:
html = res.text
len(html)

49170

In [6]:
soup = BeautifulSoup(html, "html.parser")
tag_list = soup.select('.section_episode_list a') # .select는 태그 리스트를 반환

for a_tag in tag_list:
    ep_url = a_tag["href"]
    if ep_url != "#":
        name_tag = a_tag.select(".name")[0]
        w_title = name_tag.text
        print(w_title)
        
        ep_url = "https://m.comic.naver.com" + ep_url
        ep_url = urljoin(index_page_url, ep_url)
        print(ep_url)

162. Ep. 28 최강의 희생양 (6)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=163&week=wed&listSortOrder=DESC&listPage=1
161. Ep. 28 최강의 희생양 (5)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=162&week=wed&listSortOrder=DESC&listPage=1
160. Ep. 28 최강의 희생양 (4)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=161&week=wed&listSortOrder=DESC&listPage=1
159. Ep. 28 최강의 희생양 (3)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=160&week=wed&listSortOrder=DESC&listPage=1
158. Ep. 28 최강의 희생양 (2)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=159&week=wed&listSortOrder=DESC&listPage=1
157. Ep. 28 최강의 희생양 (1)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=158&week=wed&listSortOrder=DESC&listPage=1
156. Ep. 27 읽을 수 없는 것 (4)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=157&week=wed&listSortOrder=DESC&listPage=1
155. Ep. 27 읽을 수 없는 것 (3)
https://m.comic.naver.com/webtoon/detail?titleId=747269&no=156&week=wed&listSortOrder=DESC

# 회차별 이미지를 저장할 디렉토리 생성

In [7]:
idx=1
f"{idx:03d}.jpg"

'001.jpg'

In [8]:
page_url = "https://m.comic.naver.com/webtoon/detail?titleId=747269&no=163"
res = requests.get(page_url)
html = res.text
soup = BeautifulSoup(html, "html.parser")
image_list = soup.select('.toon_view_lst p img')
len(image_list)

91

In [9]:
tag = image_list[0]
tag

<img alt="" class="fx2 lazy toon_image" data-src="https://image-comic.pstatic.net/mobilewebimg/747269/163/de52849320e7d3cf4021d98a8fff7a25_001.jpg" id="toon_0" src="https://ssl.pstatic.net/static/m/comic/im/2012/bg_transparency.png"/>

In [10]:
tag['src']

'https://ssl.pstatic.net/static/m/comic/im/2012/bg_transparency.png'

In [ ]:
import os
from tqdm.notebook import tqdm

toon_title = "전지적 독자 시점"

for a_tag in tag_list:
    ep_url = a_tag["href"]
    if ep_url != "#":
        name_tag = a_tag.select(".name")[0]
        w_title = name_tag.text
        print(w_title + " - Download start !!")
        
        ep_url = "https://comic.naver.com" + ep_url
        ep_url = urljoin(index_page_url, ep_url)
        #print(ep_url)
        
        episode_res = requests.get(ep_url, headers=headers)
        episode_html = episode_res.text
        episode_soup = BeautifulSoup(episode_html, "html.parser")
        episode_list = episode_soup.select(".wt_viewer img")
        
        for idx, img_tag in enumerate(episode_list, 1):
            filepath = f"{toon_title}/{w_title}/{idx:03d}.jpg" 
            #print(filepath)
            
            # 경로 계산하고, 없으면 생성  
            dirpath = os.path.dirname(filepath)
            os.makedirs(dirpath, exist_ok=True)

            image_url = img_tag['src']
            image_res = requests.get(img_tag["src"], headers=headers)
            #assert image_res.status_code == 200, "서버 응답이 200이 아니예요!!!"

            # 저장
            with open(filepath, "wb") as f:
                f.write(image_res.content)